In [1]:
from hud import gym
from hud.utils import stream
from hud.task import Task

### Browser Task Example

In [53]:
# Simple browser task with a single step
task = Task(
    prompt="""Given the set of past transactions (inflows into our account), project our breakeven date given our implied growth rate and our bank balance of $2,003,232 and fixed expenses of 150k per mo. 
Produce your answer as a value in the cell A1 of a sheet in the spreadsheet. Ensure there is nothing else in the answer sheet. Title this answer sheet “ANSWER”. 
You may create as many additional sheets as you need to conduct your analysis. If you build a model, put it in its own tab separate from the raw data. 

""",
    gym="hud-browser",
    setup=("goto", "https://docs.google.com/spreadsheets/d/1eCpEDv9wuVqUuCQBjC4m-LsRZJVeO869TUCPELJApTU/edit?usp=sharing"),
    evaluate=("sheet_contains", "2025-03-25")
)

In [57]:
# Create and set up environment with google, takes around 20 seconds
env = await gym.make(task)
urls = await env.get_urls()

# Stream the live view
stream(urls["live_url"])

'\n    <div style="width: 960px; height: 540px; overflow: hidden;">\n        <div style="transform: scale(0.5); transform-origin: top left;">\n            <iframe src="https://live.anchorbrowser.io?sessionId=5b23cf64-d1c6-4e37-893b-1a0e0f5c0dca" width="1920" height="1080" style="border: 1px solid #ddd;">\n            </iframe>\n        </div>\n    </div>\n    '

In [58]:
await env.evaluate(("sheet_contains", "2024-01-04"))

{'logs': None, 'error': None, 'reward': 1.0}

In [21]:
from hud.agent import ClaudeAgent, OperatorAgent

# Define a new agent each time to reset the message history
# Make sure to define the environment variable OPENAI_API_KEY
# Use Claude if you don't have access to Operator
agent = ClaudeAgent()

# Initial observation
obs, _ = await env.reset()
print(f"Initial observation complete")

# Agent loop
for i in range(50):
    print(f"========= Step {i+1} =========")

    # Use the agent to predict an action
    action, done = await agent.predict(obs)
    print(f"Agent's action (CLA): {action}")
    
    # Step the environment with the action
    obs, reward, terminated, info = await env.step(action) # TODO RESPONSE AGENT

    if terminated or done:
        break


Initial observation complete
========= Step 1 =========
Agent's action (CLA): [ScreenshotFetch(type='screenshot')]
========= Step 2 =========
Agent's action (CLA): [ClickAction(type='click', point=Point(x=261, y=881), selector=None, button='left', pattern=None, hold_keys=None)]
========= Step 3 =========
Agent's action (CLA): [ClickAction(type='click', point=Point(x=247, y=882), selector=None, button='left', pattern=None, hold_keys=None)]
========= Step 4 =========
Agent's action (CLA): [ClickAction(type='click', point=Point(x=275, y=882), selector=None, button='left', pattern=None, hold_keys=None)]
========= Step 5 =========
Agent's action (CLA): [ClickAction(type='click', point=Point(x=275, y=882), selector=None, button='left', pattern=None, hold_keys=None)]
========= Step 6 =========
Agent's action (CLA): [ClickAction(type='click', point=Point(x=261, y=881), selector=None, button='left', pattern=None, hold_keys=None)]
========= Step 7 =========
Agent's action (CLA): [ClickAction(typ

Network error:  from https://orcstaging.hud.so/hud-gym/api/v2/environments/72b82730-c470-41cd-b201-8d5234e623ca/invoke, retrying in 2.00 seconds (attempt 1/4)
HTTP error from HUD SDK: Request failed: Failed to execute step: Async passthrough error: Request failed: Step execution failed: BrowserType.connect_over_cdp: WebSocket error: wss://connect.anchorbrowser.io/ 404 Not Found
{"error":{"code":404,"message":"Invalid session"}}

Call log:
  - <ws connecting> wss://connect.anchorbrowser.io/
  -   - <ws unexpected response> wss://connect.anchorbrowser.io/ 404 Not Found
{"error":{"code":404,"message":"Invalid session"}}
  -   - <ws error> wss://connect.anchorbrowser.io/ error WebSocket was closed before the connection was established
  -   - <ws connect error> wss://connect.anchorbrowser.io/ WebSocket was closed before the connection was established
  -   - <ws disconnected> wss://connect.anchorbrowser.io/ code=1006 reason=
 | Status: 500 | Response Text: {"detail":"Step execution failed:

RequestError: Request failed: Failed to execute step: Async passthrough error: Request failed: Step execution failed: BrowserType.connect_over_cdp: WebSocket error: wss://connect.anchorbrowser.io/ 404 Not Found
{"error":{"code":404,"message":"Invalid session"}}

Call log:
  - <ws connecting> wss://connect.anchorbrowser.io/
  -   - <ws unexpected response> wss://connect.anchorbrowser.io/ 404 Not Found
{"error":{"code":404,"message":"Invalid session"}}
  -   - <ws error> wss://connect.anchorbrowser.io/ error WebSocket was closed before the connection was established
  -   - <ws connect error> wss://connect.anchorbrowser.io/ WebSocket was closed before the connection was established
  -   - <ws disconnected> wss://connect.anchorbrowser.io/ code=1006 reason=
 | Status: 500 | Response Text: {"detail":"Step execution failed: BrowserType.connect_over_cdp: WebSocket error: wss://connect.anchorbrowser.io/ 404 Not Found\n{\"error\":{\"code\":404,\"message\":\"Invalid session\"}}\n\nCall log:\n  

In [24]:
await env.evaluate()

{'logs': None, 'error': None, 'reward': 0.0}

In [6]:
await env.close()

### Response Task Example

In [ ]:
# Simple response task with a single step
task = Task(
    prompt="What is the capital of France?",
    gym="hud-browser",
    setup=("goto", "https://www.google.com"),
    evaluate=("response_includes", "Paris")
)

In [ ]:
# Create and set up environment with google, takes around 20 seconds
env = await gym.make(task)
urls = await env.get_urls()

# Stream the live view
stream(urls["live_url"])

In [ ]:
from hud.agent import ClaudeAgent, ClaudeAdapter, OperatorAgent, OperatorAdapter

# Define a new agent each time to reset the message history
# Make sure to define the environment variable OPENAI_API_KEY
agent = OperatorAgent(adapter=OperatorAdapter())

# Initial observation
obs, _ = await env.reset()
print(f"Initial observation complete")

# Agent loop
for i in range(8):
    print(f"========= Step {i+1} =========")

    # Use the agent to predict an action
    action, done = await agent.predict(obs)
    print(f"Agent's action (CLA): {action}")
    
    # Step the environment with the action
    # In this case, the final action will have to submit a response
    obs, reward, terminated, info = await env.step(action)

    if done or terminated:
        break


In [ ]:
env.final_response

In [ ]:
await env.evaluate()

### Response Task without provisioning CUA environments

In [16]:
# Response task with qa env
task = Task(
    prompt="What is the capital of France?",
    gym="qa",
    evaluate=("response_includes", "Paris")
)

In [ ]:
env = await gym.make(task)

In [ ]:
from hud.adapters.common.types import ResponseAction
await env.step([ResponseAction(text="Paris")])

In [ ]:
await env.evaluate()